### Webscraping notes
I used BeautifulSoup library for webscraping and EC2 on AWS to run it. This part actually took quite long, because I needed to account for many errors (including both request errors and issues with the entered url), so often the code would run for several hours and then break. I ended up batching it first. And then reran as a whole df.

I recorded both info from the homepage and the status errors, to consider whether the urls provided were no longer in existence. This also contributed to the "freshness" of the client company info, which I will discuss in the futher notebooks.

In [32]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError

In [33]:
df = pd.read_pickle("./clean_urls")

In [34]:
df.head()

,co_id,website
2,0012ad11-e1fb-4fa1-85fd-33bd428f0427,http://www.anovavoz.com.br
3,00150029-74d1-4040-978b-f84116ffe568,http://markiapp.com
4,0016ce4f-eee4-43e8-8913-32ce35689f0e,Https://zoisolar.com
5,0016db7b-45a5-48bb-a5d2-7157a46d283f,http://www.abi-africa.com
6,0017ede6-3113-403a-ae6f-4ec1517f2e04,http://www.jurispprudence.com


### 1. Scraping function: ping the website to check if it's still alive, scrape homepage, record error message if there is one.

The reason for recording error messages to later see whether a company's url was 403 (website no longer available) and note that those clients are potentially no longer there.

In [6]:
def scrape_soup(url):
    import idna
    try:
        res = requests.get(url)
        status_code = res.status_code
        url = res.url
        error = "No error"
        soup = (BeautifulSoup(res.content, "html5lib"))
        print(url, error, status_code)
    except requests.ConnectionError as e:
        error = ("CONNECTION ERROR: " + str(e))
        url = url
        status_code = "Error1"
        soup = "Error1"
        print(url, error, status_code)
    except idna.IDNAError as e:
        error = ("IDNA ERROR:" + str(e))
        url = url
        status_code = "Error2"
        soup = "Error2"
        print(url, error, status_code)
    except requests.exceptions.ReadTimeout as e:
        error = ("TIMEOUT ERROR:" + str(e))
        url = url
        status_code = "Error3"
        soup = "Error3"
        print(url, error, status_code)
   
    return (url, status_code, error, soup)

### 2. Scraping (note I did in batches using EC2 on AWS).

In [ ]:
# DO NOT RUN, UNLESS YOU ARE RERUNNING ON PURPOSE. IT'LL TAKE A REALLY LONG TIME.
results = df['website'].apply(scrape_soup)

In [ ]:
#split that data into status code (to see whether the website is still running) and the rest (for future processing)
df['url'] = [result[0] for result in results]
df['status_code'] = [result[1] for result in results]
df['error'] = [result[2] for result in results]
df['soup'] = [result[3] for result in results]

### 3. URLs with info from scraping

In [19]:
df_new = pd.read_pickle("./FINAL_pickle_soup")

In [23]:
df_new.status_code.value_counts()

200       6884
Error1     611
403        392
404        154
429         32
503         23
500         15
504          8
401          7
412          4
999          3
502          3
410          3
416          3
526          3
Error2       2
Error3       2
402          2
406          2
524          1
522          1
409          1
424          1
203          1
477          1
418          1
521          1
Name: status_code, dtype: int64

Out of 8K+ urls just under 7K came back with status code 200, around 400 with status code 403 and the rest with difference status code errors.